In [1]:
import requests

import numpy as np
import pandas as pd

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

pd.set_option('display.max_columns', None) # display all columns in DF
pd.set_option('display.max_rows', None) # display all columns in DF

# Data from openFDA NDC

Load data

In [2]:
df_openFDA_NDC = pd.read_csv('raw_openFDA_NDC_data.csv', sep = '~')

Clear data and remove duplicates

In [3]:
df_openFDA_NDC = df_openFDA_NDC['generic_name'].str.lower()
df_openFDA_NDC = pd.DataFrame(df_openFDA_NDC.drop_duplicates())

In [4]:
df_openFDA_NDC

,generic_name
0,mometasone furoate and dimethicone
1,desonide
2,desmopressin acetate
3,sildenafil
4,indomethacin
6,aripiprazole
9,venlafaxine
10,hydrocodone bitartrate and acetaminophen
11,lisinopril
12,glycopyrrolate


# Data from NIH

In [41]:
df_openFDA_NDC = df_openFDA_NDC[:1]

count = 0

for index, row in df_openFDA_NDC.iterrows(): # iterating through drugs
    
    try:
        count = count + 1
        
        drug_name = row[0]
        print(drug_name)
        print(count)

        start_url = 'https://rxnav.nlm.nih.gov/REST/rxclass/class/byDrugName.json?drugName='
        middle_url = str(drug_name)

        r = requests.get(start_url + middle_url, timeout = 20) # requesting meta data
        data_raw = r.json()
        
        data = data_raw['rxclassDrugInfoList']

        df_temp = pd.json_normalize(data['rxclassDrugInfo'])

        df_temp = df_temp.loc[df_temp['minConcept.name'] == drug_name]
        df_temp = df_temp.loc[(df_temp['rela'] == 'has_epc') & (df_temp['relaSource'] == 'DAILYMED')]
        
        if count == 1:
            df_NIH = df_temp
            print('Hey')
        else:
            df_NIH = pd.concat([df_NIH, df_temp])
        
    except:
        continue
        
df_NIH.head()

mometasone furoate and dimethicone
1


,rela,relaSource,minConcept.rxcui,minConcept.name,minConcept.tty,rxclassMinConceptItem.classId,rxclassMinConceptItem.className,rxclassMinConceptItem.classType,rxclassMinConceptItem.classUrl
28,has_epc,DAILYMED,3254,desonide,IN,N0000175576,Corticosteroid,EPC,NaN
28,has_epc,DAILYMED,3254,desonide,IN,N0000175576,Corticosteroid,EPC,NaN


In [15]:
drug_name = 'olanzapine' #'semaglutide' # 'olanzapine' #

start_url = 'https://rxnav.nlm.nih.gov/REST/rxclass/class/byDrugName.json?drugName='
middle_url = str(drug_name)

r = requests.get(start_url + middle_url, timeout = 20) # requesting meta data
data_raw = r.json()
data = data_raw['rxclassDrugInfoList']

df = pd.json_normalize(data['rxclassDrugInfo'])

df_drug = df.loc[df['minConcept.name'] == drug_name]
df_drug = df.loc[(df['rela'] == 'has_epc') & (df['relaSource'] == 'DAILYMED')]

#df_drug = df.loc[df['rela'] == 'has_epc']

#df = df.loc[df['rxclassMinConceptItem.classType'] == 'DISEASE']

In [16]:
df_drug

,rela,relaSource,minConcept.rxcui,minConcept.name,minConcept.tty,rxclassMinConceptItem.classId,rxclassMinConceptItem.className,rxclassMinConceptItem.classType
78,has_epc,DAILYMED,61381,olanzapine,IN,N0000175430,Atypical Antipsychotic,EPC


In [ ]:
Brug EPC

In [11]:
data

{'rxclassDrugInfoList': {'rxclassDrugInfo': [{'minConcept': {'rxcui': '1294588',
     'name': 'olanzapine pamoate',
     'tty': 'PIN'},
    'rxclassMinConceptItem': {'classId': 'D001714',
     'className': 'Bipolar Disorder',
     'classType': 'DISEASE'},
    'rela': 'may_treat',
    'relaSource': 'MEDRT'},
   {'minConcept': {'rxcui': '1294588',
     'name': 'olanzapine pamoate',
     'tty': 'PIN'},
    'rxclassMinConceptItem': {'classId': 'D011618',
     'className': 'Psychotic Disorders',
     'classType': 'DISEASE'},
    'rela': 'may_treat',
    'relaSource': 'MEDRT'},
   {'minConcept': {'rxcui': '1294588',
     'name': 'olanzapine pamoate',
     'tty': 'PIN'},
    'rxclassMinConceptItem': {'classId': 'D012559',
     'className': 'Schizophrenia',
     'classType': 'DISEASE'},
    'rela': 'may_treat',
    'relaSource': 'MEDRT'},
   {'minConcept': {'rxcui': '61381', 'name': 'olanzapine', 'tty': 'IN'},
    'rxclassMinConceptItem': {'classId': 'D001714',
     'className': 'Bipolar Disor